# Constructing aggregation matrix
`Execute notebook once:`
- Will construct 2 aggregation matrices for `totalSales` in each of `spatial` and `product` dimensions, (105x57) and (61x44) respectively.
- Will construct **combined** aggregation matrices for `totalSales` in both `spatial` and `product` dimensions (cross product terms included): (165x2217) internal nodes + (6240x2217) cross-terms = (6405x2217).
- Matrix entries are `boolean`
- Create pickle files of dataframe of each matrix

<font color="red">*** *WARNING: Does NOT overwrite****</font> if file already exists in `../data/` directory. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import pickle

import logging
logging.basicConfig(level=logging.DEBUG)

logger = logging.getLogger(__name__)
logger.debug('hello')   # Logger's method

plt.rcParams['figure.figsize'] = [16, 8]

import time


DEBUG:__main__:hello


In [2]:
# debugging function

DEBUG = False

def log(*s):
    if DEBUG:
        print(*s)

log("Debugging ON:", np.ones(2, dtype=int), ["test1", "test2"])

print("Starting...")

Starting...


In [3]:
# Loading dataset

df = pd.read_csv('~/data/dataset-02.csv.gz')
df

,time,Unnamed: 0,year,month,week,province,city,store,category,department,class,vendor,size,totalQuantity,totalSales,Holiday
0,2012-01-08,0,2012,1,1,province3,city31,store40,category1,department1,class7,vendor2,size26,1.0,424.971081,True
1,2012-01-08,1,2012,1,1,province2,city15,store33,category1,department1,class7,vendor2,size29,-1.0,-424.971081,True
2,2012-01-08,3,2012,1,1,province1,city18,store51,category1,department1,class7,vendor2,size29,1.0,424.971081,True
3,2012-01-08,4,2012,1,1,province3,city31,store62,category1,department1,class7,vendor2,size29,1.0,424.971081,True
4,2012-01-08,5,2012,1,1,province3,city31,store40,category1,department1,class7,vendor2,size33,1.0,424.971081,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2348925,2018-01-07,2639097,2018,1,1,province2,city12,store31,category2,department12,class41,vendor11,size1,2.0,85.708453,True
2348926,2018-01-07,2639098,2018,1,1,province1,city39,store43,category2,department12,class41,vendor11,size1,2.0,85.708453,True
2348927,2018-01-07,2639099,2018,1,1,province1,city23,store56,category2,department12,class41,vendor11,size1,1.0,42.854227,True
2348928,2018-01-07,2639100,2018,1,1,province1,city35,store45,category2,department12,class41,vendor84,size1,1.0,103.564381,True


## Constructing aggregation matrix

In [4]:
## Iterative implementation for space hierarchy

# deprecated in favor of generic recursive function


In [5]:
space_order = ['province', 'city', 'store']
product_order = ['category', 'department', 'class']
prefix = space_order[-1], product_order[-1]


### Refactoring into generic _methods_

In [6]:

# recursive implementation for constructing summation matrix over a given hierarchy

from scipy.linalg import block_diag # nparray
import scipy.sparse as sp # sparse matrix

def sum_group(df, columns, path=""):
    """
    Inputs
    df: dataframe of data to aggregate
    columns: list if columns from the dataframe df that represent the hierarchy to traverse (higher to lower)

    Output
    S: aggregation matrix for df over all hierarchies defined in columns (sparse matrix COO format)
        -COO is a fast format for constructing sparse matrices
        -Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector operations
        dimensions = (all combination of nodes x all leaves) in the hierarchy
        if a joint-combined hierachy is provided, futher processing is needed to groupby duplicate rows,
        use aggregate_group()
    col_names: names/labels in order the leaves(columns) are represented in S
    row_names: names/labels in order the aggregations(rows) are represented in S 
               (does not include the name of the aggregation above columns[0])
    """
    
    assert len(columns) >= 2, 'Hierarchy must contain atleast one level (parent->child)'
    
    if len(columns) == 2: # last level of hierarchy = leaf and its parent
        log('Processing Leaf')
        
        parent_field = columns[0]
        leaf_field = columns[1]
        
        submatrix = []
        col_names = []
        row_names = []
        
        for subname, subgroup in df.groupby(parent_field):
            log(parent_field, subname, subgroup[leaf_field].unique())

            leaves = subgroup[leaf_field].unique()
            
            ident = sp.eye(len(leaves), dtype=bool) # identity matrix for all the leaves
            ones = np.ones(len(leaves), dtype=bool) # row of ones to aggregate the leaves
            ones = sp.coo_matrix(ones, dtype=bool) # Sparse matrix representation of ones()

            #S = np.row_stack((ones, ident))
            S = sp.vstack([ones, ident])
            log('Number of elements in %s: %d'%(subname, S.shape[1]))

            submatrix.append(S)
            
            group_path = "+".join([path, subname])
            leaf_path = ["+".join([group_path, l]) for l in leaves]
            col_names.extend(leaf_path)
            row_names.extend([subname] + leaves.tolist())
            #row_names.extend([subname] + leaf_path) # possibly deprecated
            
            
        S = combine(submatrix) 
        log('Number of leaves in %s+%s: %d'%(path, parent_field, S.shape[1]) ) # columns of matrix
          
        return S, (row_names, col_names)

    group_on = columns[0]
    log('Processing Children of:', group_on) # Traversing hierarchy in DFS order
    
    matrix = []
    col_names = []
    row_names = []
    for name, group in df.groupby(group_on): # 0 = first level (highest) of hierarchy
        log(name, columns)
        
        group_path = "+".join([path, name])
        
        S, names = sum_group(group, columns[1:], group_path) # recurse over remaining levels in the hierarchy and retrieve their summation matrix
        
        log('Number of elements in %s: %d'%(name, S.shape[1]))
        matrix.append(S)
        
        row, col = names
        row_names.extend([name] + row)
        col_names.extend(col)
        log()
     
    S = combine(matrix)
    log('Number of nodes in %s+%s: %d'%(path, group_on, S.shape[1])) # columns of matrix
    log()
    
    assert len(col_names) == S.shape[1], 'Number of columns:%d, Columns in Matrix: %d'(len(col_names),S.shape[1])
    assert len(row_names) == S.shape[0]-1, 'Number of rows:%d, Rows in Matrix: %d'(len(col_names),S.shape[0]-1) # -1 because the top row is the aggregation of all leaves in the hierarchy tree whose name is known outside the hierachy
    
    return S, (row_names, col_names)

def combine(submatrix):
    """
    Inputs
    submatrix: list of submatrices to be used to construct the block diagonal matrix
    adds row of ones to the top to represent aggregation of submatrices
    
    Output
    S: aggregation matrix (sparse matrix COO format)
        -COO is a fast format for constructing sparse matrices
        -Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector operations
    """
    
    #S = block_diag(*matrix)
    S = sp.block_diag(tuple(submatrix)) # Sparse matrix implementation
    cols = S.shape[1]
    
    ones = np.ones(cols, dtype=bool)    
    ones = sp.coo_matrix(ones, dtype=bool) # Sparse matrix representation of ones()

    #S = np.row_stack((ones,S))
    S = sp.vstack([ones, S]) # Sparse matrix version of row_stack() 
    
    assert S.getrow(0).sum() == cols # top row must be all 1's representing aggregation of submatrix
    
    return S


In [7]:
def post_process(combined_S, grouped_names, root="sales"):
    """
    Inputs
        combined_S: Matrix to be used for aggregation that 
                    needs post-processing if a joint-combined hierarchy was used
        grouped_names: tuple(list, list), 
            grouped_names[0]: row names of the nodes enountered during forming aggregation matrix
            grouped_names[1]: column names of the leaves enountered during forming aggregation matrix
        root: str, label for the top level of aggregation

    Outputs
        combined_df: dataframe, for aggregation matrix for all nodes of hierarchy tree
                    (does not include cross-terms, use cross_term() for that)
    """
    
    names = ([root] + grouped_names[0], grouped_names[1]) # Add label for target row (row of ones)

    # combined_df = pd.DataFrame.sparse.from_spmatrix(combined_S, index=names[0], columns=names[1])
    # 'groupby' operations take too long on a Dataframe with SparseArray columns
    # using a regular dataframe for time savings at cost of speed.
   
    if sp.issparse(combined_S):
        combined_S = combined_S.tocsr()
        
    combined_df = pd.DataFrame(combined_S.todense(), index=names[0], columns=names[1])
    
    combined_df[combined_df.index.duplicated(keep=False)] # keep=False: means keep all the duplicates

    print('Unique rows in grouping:',len(combined_df.index.unique()))
    
    log('grouping...')
    t0 = time.time()
    
    combined_df = combined_df.groupby(by=combined_df.index, sort=False).agg('any') # duplicate indices are aggregated (since entries are 0/1 summing will also do)
    
    t1 = time.time()
    log('Time:', t1-t0)

    return combined_df



## *spatial* hierarchy

- **Dense** `int` matrix representation is *shape* = (105, 57), *nbytes* = 47880 ~46.88MB

- **Sparse** `int` matrix representation is *shape* = (105, 57), *nbytes* = 1824 ~1.8MB

- **Sparse** `bool` matrix representation is *shape* = (105, 57), *nbytes* = 228

- **Dataframe from dense**  memory usage: 6.7+ KB

- **Dataframe from sparse**  memory usage: 1.9 KB




In [8]:
space_order = ['province', 'city', 'store']

space_S, space_names = sum_group(df, space_order, "sales")

log('is_sparse:', sp.issparse(space_S))
    
# Regular matrix (np.array)
# space_S.shape, space_S.nbytes 

# Sparse matrix:
print('shape', space_S.shape, 'bytes', space_S.data.nbytes)

space_S.dtype

shape (105, 57) bytes 228


dtype('bool')

### Visualize *spatial* aggregation matrix

In [9]:
# Visualize space

space = post_process(space_S, space_names, "sales")
space


Unique rows in grouping: 105


,sales+province1+city11+store47,sales+province1+city11+store57,sales+province1+city13+store41,sales+province1+city18+store51,sales+province1+city20+store60,sales+province1+city23+store56,sales+province1+city3+store53,sales+province1+city30+store49,sales+province1+city35+store45,sales+province1+city37+store48,...,sales+province2+city8+store32,sales+province2+city9+store8,sales+province3+city31+store40,sales+province3+city31+store62,sales+province3+city31+store63,sales+province3+city31+store65,sales+province3+city33+store61,sales+province3+city33+store64,sales+province3+city40+store39,sales+province3+city49+store46
sales,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
province1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
city11,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
store47,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
store57,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
store64,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
city40,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
store39,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
city49,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


## *product* hierarchy

- **Dense** `int` matrix representation is *shape* = (61, 44), *nbytes* = 21472 ~21.5MB

- **Sparse** `int` matrix representation is *shape* = (61, 44), *nbytes* = 1408 ~1.4MB

- **Sparse** `bool` matrix representation is *shape* = (61, 44), *nbytes* = 176

- **Dataframe from sparse**  memory usage: 3.1+ KB




In [10]:
product_order = ['category', 'department', 'class']

prod_S, prod_names = sum_group(df, product_order, "sales")

log('is_sparse:', sp.issparse(prod_S))

# Regular matrix (np.array)
# prod_S.shape, prod_S.nbytes 

# Sparse matrix:
print('shape', prod_S.shape, 'bytes', prod_S.data.nbytes)



shape (61, 44) bytes 176


### Visualize *product* aggregation matrix

In [11]:
# Visualize product

prod = post_process(prod_S, prod_names, "sales")
prod


Unique rows in grouping: 61


,sales+category1+department1+class7,sales+category1+department1+class14,sales+category1+department1+class15,sales+category1+department2+class1,sales+category1+department2+class2,sales+category1+department2+class3,sales+category1+department2+class4,sales+category1+department2+class5,sales+category1+department2+class6,sales+category1+department3+class8,...,sales+category2+department12+class39,sales+category2+department12+class40,sales+category2+department12+class41,sales+category2+department13+class42,sales+category2+department13+class43,sales+category2+department14+class44,sales+category2+department8+class22,sales+category2+department8+class24,sales+category2+department8+class23,sales+category2+department9+class25
sales,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
category1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
department1,True,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
class7,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
class14,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
class22,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
class24,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
class23,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
department9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


## Grouped hierarchy: combined *space* and *product*



**np** matrix:
- *space*->*product*: _shape_ = (3077, 2217), _nbytes_ = n/a
- *product*->*space* _shape_ = (4145, 2217), _nbytes_ = n/a


**Sparse** matrix:
- *space*->*product*:
- - _shape_ = (3077, 2217), _nbytes_ = 124152  ~124MB (`int`) 
- - _nbytes_ = 15519 ~15MB (`bool`)
- *product*->*space* 
- - _shape_ = (4145, 2217), _nbytes_ = 124152 (`int`)
- - _nbytes_ = 15519 ~15MB (`bool`)


**Dataframe**
- - Intermediate DF (without cross-terms)
- - - *dtypes*: bool(2217), *memory usage*: 358.5+ KB
- - Final (combined with cross-terms)
- - - *dtypes*: bool(2217), *memory usage*: 13.6+ MB
- - processing time: *O(seconds)*


**Sparse** dataframe:
- - Intermediate DF (without cross-terms)
- - - *dtypes*: Sparse\[bool, False\](2217), *memory usage*: 179.8+ KB
- - - `groupby` operations take <font color="red">**too long**</font> *post_process()*: 2.5mins, *cross_terms()*: >30mins (in combined hierarchy)
- - - Did not complete computation of *cross_terms()*, took too long
- - Final (with cross-terms)
- - - *n/a*
- Switching back to implementation that uses **Dataframes**: more memory but drastic time savings


In [12]:
combo1 = space_order + product_order
combo2 = product_order + space_order

combined_S, grouped_names = sum_group(df, combo1, "sales")
log('is_sparse:', sp.issparse(combined_S))
    
# Regular matrix (np.array)
# grouped_S.shape, grouped_S.nbytes 

#Sparse matrix:
print('shape', combined_S.shape, 'bytes', combined_S.data.nbytes) 




shape (3077, 2217) bytes 15519


In [13]:
# Further process to obtain cross-terms
def cross_terms(combined_df, space_order, product_order, root="sales"):
    """
    Inputs:
        combined_df: dataframe for the nodes in each hierarchy
        space_order: list
        product_order: list
        root: str, label for the top level of aggregation

    Outputs:
        S: dataframe, aggregation matrix containing entries for all cross terms
    """

    S = combined_df.copy()

    ind = S.index.tolist()

    all_spaces = [i for i in ind for t in space_order if i.startswith(t)]
    all_products = [i for i in ind for t in product_order if i.startswith(t)]

    # Separate in multiple hierarchical groups, 'sales' row is excluded
    S.loc[all_spaces]
    S.loc[all_products]

    # cartesian product
    S['key'] = 0
    
    log('merging (cross-product) ...')
    t0 = time.time()
    S = pd.merge(S.loc[all_spaces].reset_index(), S.loc[all_products].reset_index(), 
                 suffixes=(':x',':y'), 
                 on='key',sort=False) # 5.4MB+ if S is sparse dataframe

    t1 = time.time()
    log('Time:', t1-t0)

    def remove_suffix(x):
        return x.split(":")[0]

    S = S.rename(columns = remove_suffix).drop(columns='key')

    # There exist multiple 'index' columns, one from each hierarchy group: combine using "+"

    S['index'] = S['index'].agg('+'.join, axis=1) # aggregation will apply to both columns, duplicate not removed
    final_index = pd.Index(S['index'].iloc[:,0], name="node") # new index for final matrix (0: duplicate removed)
   
    S = S.drop(columns='index')
    
    log('grouping columns...')

    # S = S.groupby(by=S.columns, axis=1).agg('any').set_index(final_index) # groupby all remaining columns(level 0, axis=1) and aggregate any ones
    # groupby rows is more efficient, this can be achived by transposing DF, groupby, then transpose result

    t0 = time.time()
    
    S = S.T
    S = S.groupby(S.index, sort=False).agg('any').astype(bool) 
    S = S.T
    
    t1 = time.time()
    log('Time:', t1-t0)

    S = S.set_index(final_index)

    return S

In [14]:
combined_df = post_process(combined_S, grouped_names, "sales")

Unique rows in grouping: 165


In [15]:

cross_df = cross_terms(combined_df, space_order, product_order, "sales")

with open('../data/cross_terms.pickle', 'rb') as f:
    valid_cross = pickle.load(f) #preprocess dataset to determine existing cross-terms: list of ('space','product')
    
valid_cross = list(map(lambda x: "+".join(x), valid_cross)) # parse tuples to match syntax 'space+product'
cross_df = cross_df.loc[valid_cross] # keep cross-terms that exist in the dataset
cross_df


,sales+province1+city11+store47+category1+department1+class7,sales+province1+city11+store47+category1+department1+class14,sales+province1+city11+store47+category1+department1+class15,sales+province1+city11+store47+category1+department2+class1,sales+province1+city11+store47+category1+department2+class2,sales+province1+city11+store47+category1+department2+class3,sales+province1+city11+store47+category1+department2+class4,sales+province1+city11+store47+category1+department2+class5,sales+province1+city11+store47+category1+department2+class6,sales+province1+city11+store47+category1+department3+class8,...,sales+province3+city49+store46+category2+department12+class37,sales+province3+city49+store46+category2+department12+class39,sales+province3+city49+store46+category2+department12+class40,sales+province3+city49+store46+category2+department12+class41,sales+province3+city49+store46+category2+department12+class38,sales+province3+city49+store46+category2+department12+class36,sales+province3+city49+store46+category2+department13+class42,sales+province3+city49+store46+category2+department8+class22,sales+province3+city49+store46+category2+department8+class24,sales+province3+city49+store46+category2+department9+class25
node,,,,,,,,,,,,,,,,,,,,,
province1+category1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
province1+category2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
province2+category1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
province2+category2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
province3+category1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
store9+class5,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
store9+class6,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
store9+class7,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
agg_combo = pd.concat([combined_df, cross_df])

def parse_columns(colname):
    """
    Renames column by removing path information and retaining 
    only the terminal nodes. Extracting those elements in the path 
    that belong to terminal nodes in the hierarchical order. Exploits 
    the fact that that terminal nodes have the same prefix.
    """
    
    leaves = [node for node in colname.split("+") if node.startswith(prefix)]
    return "+".join(leaves)
    
agg_combo = agg_combo.rename(columns = parse_columns)

agg_combo


,store47+class7,store47+class14,store47+class15,store47+class1,store47+class2,store47+class3,store47+class4,store47+class5,store47+class6,store47+class8,...,store46+class37,store46+class39,store46+class40,store46+class41,store46+class38,store46+class36,store46+class42,store46+class22,store46+class24,store46+class25
sales,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
province1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
city11,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
store47,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
category1,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
store9+class5,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
store9+class6,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
store9+class7,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
store9+class8,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


### Save variables

In [17]:
import os.path

# Does not overwrite

if not os.path.isfile('../data/agg_space.pickle'):
    space.to_pickle('../data/agg_space.pickle')

if not os.path.isfile('../data/agg_prod.pickle'):
    prod.to_pickle('../data/agg_prod.pickle')
    
if not os.path.isfile('../data/agg_combo.pickle'):
    agg_combo.to_pickle('../data/agg_combo.pickle')


In [18]:
%%save_read_vars # hack. Will stop execution of this cell, comment if execution desired

import pickle

# Save Variable
df.to_pickle('../data/my_df.pickle')

with open('../data/agg_space.pickle', 'wb') as f:
    pickle.dump(some_obj, f)
    
with open('../data/agg_product.pickle', 'wb') as f:
    pickle.dump(some_obj, f)

# Read Variable
with open('../data/agg_space.pickle', 'rb') as f:
    some_obj = pickle.load(f)
    
with open('../data/agg_product.pickle', 'rb') as f:
    some_obj = pickle.load(f)
    
pd.read_pickle('../data/my_df.pickle')


UsageError: Cell magic `%%save_read_vars` not found.
